In [1]:
import os
from tqdm import tqdm
# from docx import Document
import docx
from langchain_core.documents import Document
from collections import Counter
import time
from llmsherpa.readers import LayoutPDFReader
from transformers import GenerationConfig, TextStreamer
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaConfig
import torch
from sentence_transformers import SentenceTransformer, util
from FlagEmbedding import FlagReranker
from unstructured.partition.docx import partition_docx
from unstructured.chunking.title import chunk_by_title
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader
from llama_index.readers.smart_pdf_loader import SmartPDFLoader
from llama_index.core.evaluation import (
    BatchEvalRunner,
    CorrectnessEvaluator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)
import asyncio
import pandas as pd
import nest_asyncio
from tqdm.asyncio import tqdm_asyncio
nest_asyncio.apply()
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

/home/przxct/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llmsherpa_api_url = "http://localhost:5010/api/parseDocument?renderFormat=all"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)

In [3]:
def chunking(file_path):
    pdf_url = file_path # also allowed is a file path e.g. /home/downloads/xyz.pdf
    doc = pdf_reader.read_pdf(pdf_url)
    return [Document(page_content=chunk.to_context_text(), metadata={}) for chunk in doc.chunks()]

In [5]:
def get_chunks(data_dir):
    chunks = []
    filenames = os.listdir(data_dir)
    
    for id in tqdm(range(0, len(filenames))):
        filename = filenames[id]
        file_path = os.path.join(data_dir, filename)
        
        if not file_path.endswith('.docx'):
            continue
        
        # doc = Document(file_path)
        # doc = docx.Document(file_path)
        # elements = partition_docx(filename=file_path)
        # count_each_type(elements)
        
        # chunk = chunk_by_title(elements)
        # chunk = chunking(elements)
        chunk = chunking(file_path)
        
        # texts = get_texts(doc)
        
        chunks.extend(chunk)
        # chunks.extend(elements)
    
    return chunks
        

In [6]:
data_dir = '/mnt/86A66D38A66D2A3F/Przxct/AI Project/LLM_raw/data/'

chunks = get_chunks(data_dir)

# print(get_chunks(data_dir))

100%|██████████| 9/9 [00:02<00:00,  3.26it/s]


In [7]:
for i, chunk in enumerate(chunks):
    print(f"================ chunk {i} ======================")
    print(chunk)

================ chunk 0 ======================
page_content='
 | QUỐC HỘI
------- | CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
Độc lập - Tự do - Hạnh phúc 
---------------
 | --- | ---
 | Luật số: 35/2024/QH15 | Hà Nội, ngày 27 tháng 6 năm 2024
'
================ chunk 1 ======================
page_content='ĐƯỜNG BỘ
Căn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam;'
================ chunk 2 ======================
page_content='ĐƯỜNG BỘ
Quốc hội ban hành Luật Đường bộ.'
================ chunk 3 ======================
page_content='NHỮNG QUY ĐỊNH CHUNG
Điều 1.
Phạm vi điều chỉnh'
================ chunk 4 ======================
page_content='NHỮNG QUY ĐỊNH CHUNG
Luật này quy định về hoạt động đường bộ và quản lý nhà nước về hoạt động đường bộ.'
================ chunk 5 ======================
page_content='NHỮNG QUY ĐỊNH CHUNG
Điều 2.
Giải thích từ ngữ'
================ chunk 6 ======================
page_content='NHỮNG QUY ĐỊNH CHUNG
Trong Luật này, các từ ngữ dưới đây được hiểu như sau:

In [7]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer , AutoModelForCausalLM , pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain . memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader , TextLoader
from langchain . chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [9]:
store = LocalFileStore("./cache/")

embed_model_id = 'keepitreal/vietnamese-sbert'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(chunks, embedder)

In [10]:
from langchain.chains import create_retrieval_chain

vector_db = Chroma.from_documents(documents = chunks,embedding = embedder)
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

In [73]:
result = retriever.invoke("vượt đèn đỏ phạt bao nhiêu?")
print("Number of relevant documents : ", len(result))

Number of relevant documents :  20


In [12]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# from apikeys import Cohere_API

reranker = HuggingFaceCrossEncoder(model_name="namdp-ptit/ViRanker")
compressor = CrossEncoderReranker(model=reranker, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [75]:
compressed_docs = compression_retriever.invoke("vượt đèn đỏ bị phạt bao nhiêu")
compressed_docs

[Document(metadata={}, page_content='Chương III\n8. Phạt tiền từ 3.000.000 đồng đến 5.000.000 đồng đối với người điều khiển xe ô tô, các loại xe tương tự xe ô tô vượt rào chắn đường ngang, cầu chung khi chắn đang dịch chuyển; vượt đường ngang, cầu chung khi đèn đỏ đã bật sáng; không chấp hành hiệu lệnh, chỉ dẫn của nhân viên gác đường ngang, cầu chung khi đi qua đường ngang, cầu chung.'),
 Document(metadata={}, page_content='Chương III\n7. Phạt tiền từ 1.000.000 đồng đến 2.000.000 đồng đối với người điều khiển máy kéo, xe máy chuyên dùng vượt rào chắn đường ngang, cầu chung khi chắn đang dịch chuyển; vượt đường ngang, cầu chung khi đèn đỏ đã bật sáng; không chấp hành hiệu lệnh, chỉ dẫn của nhân viên gác đường ngang, cầu chung khi đi qua đường ngang, cầu chung.'),
 Document(metadata={}, page_content='Chương III\n5. Phạt tiền từ 600.000 đồng đến 1.000.000 đồng đối với người điều khiển xe mô tô, xe gắn máy (kể cả xe máy điện), các loại xe tương tự xe mô tô và các loại xe tương tự xe gắn m

In [40]:
torch_dtype = torch.bfloat16
model_id = "llm4fun/vietrag-7b-v1.0"
device = "cuda"

tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(
    model_id,
    config=LlamaConfig.from_pretrained(model_id),
    torch_dtype=torch_dtype
)

# model = model.eval().to(device)
model = model.eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]


In [41]:
import torch
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

In [42]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    temperature=0.0,
    do_sample=False,
    max_new_tokens=1024
)

In [43]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [44]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate


def get_prompt(question, contexts):
    context = "\n\n".join([f"Context [{i+1}]: {x}" for i, x in enumerate(contexts)])
    # context = " ".join(contexts)
    instruction = 'You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.'
    # instruction = 'As an intelligent AI model, your task is to analyze and integrate information from multiple contexts given below in order to answer questions and provide citations.'
    input = f"Dựa vào một số ngữ cảnh được cho dưới đây, trả lời câu hỏi ở cuối.\n\n{context}\n\nQuestion: {question}\nHãy trả lời chi tiết và đầy đủ."
    prompt = prompt_template.format(
        instruction=instruction,
        input=input,
        output=''
    )
    return prompt

In [45]:
import json
from fastapi import FastAPI
from fastapi.responses import StreamingResponse, FileResponse
from fastapi.middleware.cors import CORSMiddleware
from langchain_core.messages import AIMessageChunk
#from logging import logging

from dotenv import load_dotenv
import bs4

In [55]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tracers.log_stream import LogEntry, LogStreamCallbackHandler
import logging

def format_docs(contexts):
    return "\n\n".join([f"Context [{i+1}]: {x}" for i, x in enumerate(contexts)])


prompt_template = """
You are an AI assistant. Provide a detailed answer so the user doesn't need to search outside to understand the answer.

Dựa vào một số ngữ cảnh được cho dưới đây, trả lời câu hỏi ở cuối.

{context}

Question: {question}

Hãy trả lời chi tiết và đầy đủ.
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],  # Đặt các biến mà template sẽ nhận
    template=prompt_template
)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": compression_retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
output = rag_chain_with_source.invoke("vượt đèn đỏ bị phạt bao nhiêu đối với xe máy?")
output